In [10]:
import os
# from dotenv import load_dotenv


from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
# from langchain_community.llms import Ollama
# from langchain_community.embeddings import OllamaEmbeddings

# MODEL="gpt-3.5-turbo"
# MODEL="llama2"
# model = Ollama(model=MODEL)
# embeddings = OllamaEmbeddings()
MODEL="gpt-4"
model = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model=MODEL)
embeddings = OpenAIEmbeddings()


In [11]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [12]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('agent-dojo.pdf')
pages = loader.load_and_split()
pages

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-11-26T02:02:21+00:00', 'author': '', 'keywords': '', 'moddate': '2024-11-26T02:02:21+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'agent-dojo.pdf', 'total_pages': 26, 'page': 0, 'page_label': '1'}, page_content='AgentDojo: A Dynamic Environment to Evaluate\nPrompt Injection Attacks and Defenses\nfor LLM Agents\nEdoardo Debenedetti1∗ Jie Zhang1 Mislav Balunovic1,2\nLuca Beurer-Kellner1,2 Marc Fischer1,2 Florian Tramèr1\n1ETH Zurich 2Invariant Labs\nAbstract\nAI agents aim to solve complex tasks by combining text-based reasoning with\nexternal tool calls. Unfortunately, AI agents are vulnerable to prompt injection\nattacks where data returned by external tools hijacks the agent to execute malicious\ntasks. To measure the adversarial robustness of AI agent

In [13]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [14]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

In [16]:
retriever = vectorstore.as_retriever()

retriever.invoke("Agents")

from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model 
    | parser
)

chain.invoke({"question": "what is this paper about?, Give me a small description"})

'The paper appears to be a study on the utility and robustness of tool-calling agents against prompt injection attacks in the field of machine learning and large language models. The document also details potential impacts, limitations of work, and ethical considerations surrounding the research. Furthermore, the authors have conducted tests and even created a dataset named "AgentDojo" for research use. The document follows an evaluative checklist to ensure comprehensive discussion on the topic.'

In [17]:
questions = [
    "What is the crux of this paper?",
    "What tool is being discussed in the paper?",
    "What is the conclusion about security against prompt injection attacks?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the crux of this paper?
Answer: The crux of this paper appears to be about developing and evaluating a work with clear guidelines for accuracy, limitation, societal impact and conformation to ethical guidelines. It also discusses the inclusion of theoretical results and stipulations for experiments, such as code, data, training details, error bars and resource usage. The paper also touches on the use of existing assets and the citation and licensing of such assets, any new assets and the nuances of using real-world data. The paper does not seem to involve working with human subjects as all data used is stated to be dummy data. The paper also suggests that there could be an extension for the work to support multimodal agents to process both text and images.

Question: What tool is being discussed in the paper?
Answer: The paper discusses a framework called "AgentDojo". It is an agent evaluation framework used to assess prompt injection attacks and defenses.

Question: 

In [ ]:
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric
from deepeval.test_case import LLMTestCase
from deepeval import evaluate

# Initialize the metric
answerRelevancy = AnswerRelevancyMetric()
# faithfulness = FaithfulnessMetric()


question = "How does the Happy Prince help the poor in the city?"
retrieved_docs = retriever.get_relevant_documents(question)
retrieved_texts = [doc.page_content for doc in retrieved_docs]
generated_answer = chain.invoke({'question': question})

test_case = LLMTestCase(
    input=question,
    actual_output=generated_answer,   # this is the LLM output
    retrieval_context=retrieved_texts
)

evaluate([test_case], [AnswerRelevancyMetric()])

# Prepare test cases
# test_cases = [
#     LLMTestCase(
#         input="What is the story about?",
#         actual_output=retriever.invoke({"question": "What is the story about"}),
#         retrieval_context=["Once upon a time, there was a prince who gave away his riches..."]
#     ),
#     # Add more test cases as needed
# ]

# # Evaluate the test cases
# evaluate(test_cases, [answerRelevancy, faithfulness])


/tmp/ipykernel_3745/1948284525.py:11: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)


✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:05,  5.45s/test case]

False !!!!!!!!!!!!


Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because the response perfectly addresses how the Happy Prince helps the poor in the city, with no irrelevant statements present. Great job on staying focused and relevant!, error: None)

For test case:

  - input: How does the Happy Prince help the poor in the city?
  - actual output: The Happy Prince helps the poor in the city by asking a swallow to take off the gold leaves that cover him, leaf by leaf, and give them to the poor. The gold leaves bring some relief to the poor, bringing color back to the children's faces and allowing them to have bread. The Prince also sends a ruby from his sword-hilt to a poor seamstress who couldn't afford oranges for her ill son.
  - expected output: None
  - context: None
  - retrieval context: ['“The ruby has fallen out of his sword, his eyes are gone, and he is\ngolden no longer,” said the May

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because the response perfectly addresses how the Happy Prince helps the poor in the city, with no irrelevant statements present. Great job on staying focused and relevant!', strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.0054275, verbose_logs='Statements:\n[\n    "The Happy Prince helps the poor in the city.",\n    "A swallow takes off the gold leaves that cover the Happy Prince.",\n    "The gold leaves are given to the poor.",\n    "The gold leaves bring relief to the poor.",\n    "The gold leaves bring color back to the children\'s faces.",\n    "The gold leaves allow children to have bread.",\n    "The Prince sends a ruby from his sword-hilt to a poor seamstress.",\n    "The seamstress couldn\'t afford oranges for her ill son."\n] \n \nVerdicts:\n[\n    {\n        "